In [ ]:
# 디베이트 전까지 테스트

In [ ]:
# 맨 위 셀에 넣기
import sys
from pathlib import Path

# capstone_project 폴더를 찾아 sys.path 맨 앞에 추가
p = Path.cwd()
root = None
for q in [p, *p.parents]:
    if q.name == "capstone_project":
        root = q
        break

if root is None:
    # 노트북이 전혀 다른 곳에서 열렸다면 경로를 직접 지정
    root = Path(r"C:\Users\ska69\capstone_project")

sys.path.insert(0, str(root))
print("PROJECT ROOT:", root)

# 테스트
import prompts
from agents.technical_agent import TechnicalAgent
print("imports ok")


In [ ]:
import os

os.environ["CAPSTONE_OPENAI_API"] = ""


In [ ]:
# 0) 경로 가드
import sys, os
print("PYTHONPATH head:", sys.path[:2])

In [ ]:
from agents.technical_agent import TechnicalAgent
import torch, json, os

# 1. 에이전트 생성
agent = TechnicalAgent(agent_id = "TechnicalAgent", ticker="NVDA", verbose=True)


In [ ]:
agent

In [ ]:
# 2. 데이터 수집 및 지표 계산(5개년) → yfinance에서 5년 데이터 다시 받음 + 13개 TECH 지표 생성
X = agent.searcher(rebuild=True)

In [ ]:
X.shape

In [ ]:
# 3. 모델 학습
# 5년 데이터 기반 LSTM 학습 + 스케일러 저장 + 가중치 저장
agent.pretrain()

In [ ]:
# 4. 최신 윈도우 확보 → 예측 → 설명 생성 → 저장
from config.agents import dir_info

# 최신 윈도우
X_last = agent.searcher(agent.ticker)      # (1,T,F)
dates  = getattr(agent.stockdata, f"{agent.agent_id}_dates", [])


In [ ]:
print(X_last.shape)
print(dates[:5])

In [ ]:
from core.technical_classes.technical_base_agent import Target
print(Target.__annotations__)

In [ ]:
# 5) 예측 + 불확실성
tgt = agent.predict(X_last) 

In [ ]:
# 6) 저장
from core.technical_classes.technical_base_agent import save_explain_json, load_explain_json

save_explain_json(agent.ticker, agent.agent_id, tgt)

In [ ]:
# 6. LLM 메시지 구성 (system, user)
sys_text, user_text = agent._build_messages_opinion(agent.stockdata, tgt)
print(sys_text[:400], "\n---\n", user_text[:700])

In [ ]:
# 7. LLM 호출 및 응답 확인
op = agent.reviewer_draft(agent.stockdata, tgt)
print(f"예측 종가: {op.target.next_close:.4f}")
print("설명:", op.reason)

In [ ]:
# debate agent 포함 test

In [ ]:
# 맨 위 셀에 넣기
import sys
from pathlib import Path

# capstone_project 폴더를 찾아 sys.path 맨 앞에 추가
p = Path.cwd()
root = None
for q in [p, *p.parents]:
    if q.name == "capstone_project":
        root = q
        break

if root is None:
    # 노트북이 전혀 다른 곳에서 열렸다면 경로를 직접 지정
    root = Path(r"C:\Users\ska69\capstone_project")

sys.path.insert(0, str(root))
print("PROJECT ROOT:", root)

# 테스트
import prompts
from agents.technical_agent import TechnicalAgent
print("imports ok")


In [ ]:
import os, sys, types
proj = r"C:\Users\ska69\capstone_project"  # 본인 경로
if proj not in sys.path:
    sys.path.insert(0, proj)

# macro_llm가 요구하는 agents.dump_keys 더미 모듈 주입
m = types.ModuleType("agents.dump_keys") #에러가 났어서 임의로 설정(아연)
m.CAPSTONE_OPENAI_API = ""  # 키 비워두면 됨. Macro를 호출하지 않을 것이라 문제 없음.
sys.modules["agents.dump_keys"] = m

In [ ]:
import os

os.environ["CAPSTONE_OPENAI_API"] = ""


In [ ]:
# 우선 테크니컬 debatagent 잘 돌아가는지 확인

from agents.debate_agent import DebateAgent

ticker = "NVDA"
da = DebateAgent(rounds=0, ticker=ticker)

USE_MACRO = False   # 매크로 포함 테스트
TECH_ONLY = True  # 테크니컬만 테스트하려면 True로

if TECH_ONLY:
    da.agents = {"TechnicalAgent": da.agents["TechnicalAgent"]}
elif USE_MACRO:
    da.agents = {
        "TechnicalAgent": da.agents["TechnicalAgent"],
        "MacroSentiAgent": da.agents["MacroSentiAgent"],
    }
else:
    da.agents = {"TechnicalAgent": da.agents["TechnicalAgent"]}

list(da.agents.keys())


In [ ]:
# 테크니컬 에이전트만 확인

ops = da.get_opinion(
    round=0,
    ticker=ticker,
    rebuild=True,          # 데이터 재빌드
    force_pretrain=True   # 가중치 없으면 내부 pretrain 실행
)
ops.keys()

In [ ]:
def show(op):
    print("agent_id:", op.agent_id)
    print("next_close:", float(op.target.next_close))
    print("uncertainty:", float(op.target.uncertainty))
    print("confidence:", float(op.target.confidence))
    print("\nreason:\n", op.reason)

if "TechnicalAgent" in ops:
    print("=== Technical ===")
    show(ops["TechnicalAgent"])


In [ ]:
ta = da.agents["TechnicalAgent"]
print(ta)

op = da.opinions[0]["TechnicalAgent"]
print(f"예측 종가: {op.target.next_close:.4f}")
print("설명:", op.reason)